In [2]:
import pemi
import pemi.data

import pandas as pd
import dask
import dask.dot

In [ ]:
## I think this structure could work and would be an improvement.
# - No need to define separate ScenarioConfig classes (which weren't clear that you needed them)
# - Makes it more explicit what you need to define a scenario
# - Could be used to automock all pipes attached to sources/targets that aren't self
# - Separating into Factories and collectors is cleaner that the poorly-specified case-collector concept



# What about mocking pipes?  Should that be done automatically, or configurably?
with Scenario(
    name='must be unique?',
    pipe=pipe,
    runner=pipe.flow,
    factories={
        'students': StudentFactory,
        'classes': ClassFactory
    },
    sources={
        'main': pipe.sources['main'],
        'student_ids': pipe.pipes['student_ids'].targets['main']
    },
    targets={
        'main': pipe.targets['main'],
        'create_students': pipe.pipes['create_students'].sources['main']
    },    
    target_case_collectors={
        'main': CaseCollector(subject_field='student_id', factory=factories['students'], factory_field='external_id')
    }
) as scenario:
    
    ex_student_ids = pemi.data.Table(
        '''
        | external_id | uuid         | merged_into_student_id |
        | -           | -            | -                      |
        | {stu[1]}    | {stuUUID[1]} |                        |
        | {stu[2]}    | {stuUUID[2]} |                        |
        '''.format(
            stu=scenario.factories['students'].field('student_id') #self.scenario.case_keys.cache('student_ids', 'student_id')
            stuUUID=scenario.factories['students'].field('student_uuid')
        ),
        schema=scenario.sources['student_ids'].schema#self.pipe.pipes['lkp_student_uuids'].pipes['student_ids'].targets['main'].schema
    )

    
    ### Or what about?
    ex_student_ids = pemi.data.Table(
        '''
        | external_id           | uuid           | merged_into_student_id |
        | -                     | -              | -                      |
        | {stu[1][external_id]} | {stu[1][uuid]} |                        |
        | {stu[2][external_id]} | {stu[2][uuid]} |                        |
        '''.format(
            stu=scenario.factories['students']
        ),
        schema=scenario.sources['student_ids'].schema#self.pipe.pipes['lkp_student_uuids'].pipes['student_ids'].targets['main'].schema
    )
    
    
    background = [
        pt.when.example_for_source(scenario.sources['student_ids'], ex_student_ids),
    ]


In [22]:
import importlib
import tests.test_noodling as t
importlib.reload(tests.test_noodling)


pipe = t.StudentClassPipe()

pipe.sources['main'].df = pemi.data.Table(
    schema=t.SOURCE_SCHEMA
).df

pipe.flow()

In [24]:
class StudentFactory():
    student_id = Lazy(pemi.data.UniqueIdGenerator('stu{}'.format))
    student_uuid = Lazy(pemi.data.UniqueIdGenerator('stu{}'.format))
    
    external_id = Lazy(student_id)
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-24-386ff8a2899e>, line 1)

In [ ]:
# How do I take something like this:
def case_keys():
    student_uuids = pemi.data.UniqueIdGenerator('stuUUID{}'.format)
    student_ids = pemi.data.UniqueIdGenerator('stu{}'.format)
    emails = pemi.data.UniqueIdGenerator('someone{}@example.com'.format)

    while True:
        student_id = next(student_ids)
        student_uuid = next(student_uuids)
        email = next(emails)

        yield {
            'main_source': {'student_id': student_id},
            'student_ids': {'student_id': student_id, 'student_uuid': student_uuid},
            'student_emails': {'student_uuid': student_uuid, 'email': email},

            'create_students': {'external_id': student_id},
            'update_students': {'student_uuid': student_uuid, 'external_id': student_id},
            'create_profiles': {'student_uuid': student_uuid, 'student_id': student_id},
            'update_profiles': {'student_uuid': student_uuid, 'student_id': student_id},
            'id_to_uuid': {'student_id': student_id, 'student_uuid': student_uuid}
        }

# and this

ex_student_ids = pemi.data.Table(
    '''
    | external_id | uuid         | merged_into_student_id |
    | -           | -            | -                      |
    | {stu[1]}    | {stuUUID[1]} |                        |
    | {stu[2]}    | {stuUUID[2]} |                        |
    '''.format(
        stu=self.scenario.case_keys.cache('student_ids', 'student_id'),
        stuUUID=self.scenario.case_keys.cache('student_ids', 'student_uuid')
    ),
    schema=self.pipe.pipes['lkp_student_uuids'].pipes['student_ids'].targets['main'].schema
)

ex_student_emails = pemi.data.Table(
    '''
    | student_uuid | email      |
    | -            | -          |
    | {stuUUID[1]} | {email[1]} |
    | {stuUUID[2]} | {email[2]} |
    '''.format(
        stuUUID=self.scenario.case_keys.cache('student_emails', 'student_uuid'),
        email=self.scenario.case_keys.cache('student_emails', 'email'),
    ),
    schema=self.pipe.pipes['student_emails'].targets['main'].schema
)

ex_coach_distribution = pemi.data.Table(
    '''
    | coach_group | coach_uuid | weight |
    | -           | -          | -      |
    | default     | coachUUID1 | 1      |
    '''
)

# and turn it into some kind of factory

In [ ]:
# I would have to mock out in some way:
# Any pipe sources
# Any targets of nested impure pipes

# We're usually testing the relationship between pipe sources/impure nested pipe targets, and impure nested pipe sources, or pipe targets

In [23]:
# I'm going to need to mock impure pipes
# If a pipe has any nested impure pipes, it is by definition impure
#   Would I always want to mock the 

# I could either keep the default to mock out the whole impure pipe
# Or I could keep the default to traverse down the nesting and only mock the impure parts

,error
0,Some random student load error
1,Some random student load error
2,Some random student load error
3,Some random student load error
4,Some random student load error
5,Some random student load error
6,Some random student load error
7,Some random student load error
8,Some random student load error
9,Some random student load error


In [17]:
pd.DataFrame({
    'class_id': list(range(100)),
    'class_uuid': [uuid.uuid4() for _ in range(100)]
})


,class_id,class_uuid
0,0,fb25e828-98f3-4ed0-b927-407f70b29037
1,1,1b8e72aa-3697-4343-a73b-781a41bde985
2,2,84ebb019-ad26-467a-8e7a-ee2da8587e01
3,3,80fa6dda-5414-4717-ab89-2b3df8eca761
4,4,c3fc196a-06b4-422e-aa4a-e04ac1afd0fb
5,5,531eec71-2cba-44c5-af9f-1ad90e96820a
6,6,aeae10d1-b1eb-4393-8f96-70b7b93d2550
7,7,31d415a2-087c-49a2-ae55-897bd41e3324
8,8,7e78c0ec-5886-41d1-a635-c8e2fb5571e6
9,9,0697311b-eab7-4130-b1a1-f9b9209f4281
